### Importing libraries...

In [13]:
import requests
from pymongo import MongoClient
import pandas as pd
import re
import json

### Importing Companies Data

In [14]:
dbName = "companies.json"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

### Verifying amount of companies of potential chosen country

In [42]:
data = db.companies.aggregate([{"$group":{"_id":"$offices.country_code","count":{"$sum":1}}}])
countries = list(data)
countries = countries[0]
len(countries)

705

In [47]:
county_counts=[]
for x in countries:
    county_counts.append(x['_id'])
county_counts = [c for n in county_counts for c in n]
county_counts

['USA',
 'IND',
 'GBR',
 'DEU',
 'USA',
 'ISR',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'FRA',
 'FRA',
 'MUS',
 'GBR',
 'USA',
 'AUS',
 'ESP',
 'NLD',
 'HKG',
 'CHN',
 'CHN',
 'TUN',
 'IND',
 'IND',
 'IND',
 'IND',
 'USA',
 'USA',
 'USA',
 'USA',
 'IND',
 'JPN',
 'GBR',
 'JOR',
 'CAN',
 'SGP',
 'USA',
 'USA',
 'GBR',
 'CHE',
 'JPN',
 'GBR',
 'IND',
 'DEU',
 'TWN',
 'BEL',
 'GBR',
 'GBR',
 'GBR',
 'DEU',
 'MAR',
 'NLD',
 'GRC',
 'CAN',
 'IND',
 'BIH',
 'USA',
 'TWN',
 'JPN',
 'KOR',
 'ARE',
 'USA',
 'IND',
 'IND',
 'USA',
 'CZE',
 'USA',
 'ESP',
 'BEL',
 'BEL',
 'DNK',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'NLD',
 'GBR',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'GBR',
 'USA',
 'USA',
 'USA',
 'GBR',
 'USA',
 'DEU',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'GBR',
 'SGP',
 'USA',
 'GBR',
 'LKA',
 'PRT',
 'PRT',
 'USA',
 'SWE',
 'USA',
 'FRA',
 'KOR',
 'USA',
 'CZE',
 'USA',
 'USA',
 'FRA',
 'USA',
 'USA',
 'FRA',
 'SWE',
 'NOR',
 'NLD',
 'SWE',
 'USA',


In [51]:
from collections import Counter
county_counts = Counter(county_counts)
{k: v for k, v in sorted(county_counts.items(), key=lambda item: item[1], reverse=True)}

{'USA': 775,
 'GBR': 257,
 'DEU': 116,
 'IND': 106,
 'FRA': 82,
 'AUS': 80,
 'CHN': 79,
 'ESP': 62,
 'CAN': 62,
 'JPN': 52,
 'SGP': 46,
 'NLD': 40,
 'SWE': 35,
 'CHE': 31,
 'BRA': 30,
 'ARG': 29,
 'ISR': 28,
 'ITA': 28,
 'HKG': 25,
 'IRL': 25,
 'RUS': 23,
 'TWN': 20,
 'BEL': 19,
 'MEX': 18,
 'NOR': 17,
 'DNK': 16,
 'KOR': 15,
 'NZL': 15,
 'ROM': 15,
 'ZAF': 15,
 'PHL': 15,
 'ARE': 14,
 'MYS': 14,
 'AUT': 14,
 'POL': 13,
 'PRT': 12,
 'FIN': 12,
 'UKR': 12,
 'CZE': 11,
 'TUR': 11,
 'CHL': 9,
 'VNM': 9,
 'BGR': 8,
 'GRC': 7,
 'THA': 6,
 'SVN': 6,
 'COL': 6,
 'EST': 5,
 'ANT': 5,
 'HUN': 5,
 'EGY': 5,
 'LKA': 4,
 'LVA': 4,
 'HRV': 4,
 'PAK': 4,
 'CYP': 4,
 'JOR': 3,
 'MAR': 3,
 'SAU': 3,
 'LTU': 3,
 'URY': 3,
 'NGA': 3,
 'BGD': 3,
 'IDN': 3,
 'MKD': 3,
 'LUX': 3,
 'MLT': 3,
 'MUS': 2,
 'BIH': 2,
 'LBN': 2,
 'ISL': 2,
 'GIB': 2,
 'PAN': 2,
 'VEN': 2,
 'PRK': 2,
 'CSS': 2,
 'NIC': 2,
 'ARA': 2,
 'VGB': 2,
 'KEN': 2,
 'QAT': 2,
 'UZB': 2,
 'TUN': 1,
 'DOM': 1,
 'SDN': 1,
 'PER': 1,
 'GHA': 1,

**Countries which companies I have more information of:**  
  
USA 🇺🇸  
GBR 🇬🇧  
DEU 🇩🇪   
IND 🇮🇪  
FRA 🇫🇷  

**Top Videogame development Countries:**

USA 🇺🇸 = 194  
GBR 🇬🇧 =  48  
DEU 🇩🇪 =   /  
IND 🇮🇪  =   /  
FRA 🇫🇷 =  23  

**Chosen Country:  
US** 🇺🇸 🇺🇸 🇺🇸 🇺🇸

## Meetup API

In [53]:
def getFromMeetup(endpoint, queryParams={}):
    url = f"https://api.meetup.com{endpoint}"
    apiKey = os.getenv("GITHUB_APIKEY")
res = requests.get(url)
res

<Response [404]>

In [ ]:
data = res.json()